In [1]:
print("OK")

OK


In [2]:
%pwd

'/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/research'

In [12]:
import os
import langchain_core
print(dir(langchain_core))

['VERSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_api', '_import_utils', 'agents', 'caches', 'callbacks', 'chat_history', 'documents', 'embeddings', 'env', 'exceptions', 'globals', 'language_models', 'load', 'messages', 'output_parsers', 'outputs', 'prompt_values', 'prompts', 'rate_limiters', 'retrievers', 'runnables', 'stores', 'surface_langchain_beta_warnings', 'surface_langchain_deprecation_warnings', 'tools', 'tracers', 'utils', 'vectorstores', 'version']


In [13]:
os.chdir('/Users/nihal/Downloads/End-End-MediRAG-GEN-AI')

In [14]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  

In [15]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)  # it extracts info using this and using glob="*.pdf" it only loads pdf files and later it gives docs

    documents = loader.load()

    return documents

In [16]:
# Check if Data directory exists, if not, use the correct path
import os
data_path = 'Data/' if os.path.exists('Data/') else '/Users/nihal/Downloads/End-End-MediRAG-GEN-AI/Data/'
extracted_data = load_pdf_file(data=data_path)


In [5]:
#extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    for i, d in enumerate(text_chunks):
        meta = d.metadata or {}

        # doc name from source path (by default pypdfloader usually sets "source")
        src = meta.get("source", "")
        meta["doc_name"] = os.path.basename(src) if src else meta.get("doc_name", "document")

        # page is typically 0 based in loaders we convert to 1 based display
        if "page" in meta and isinstance(meta["page"], int):
            meta["page_display"] = meta["page"] + 1
        else:
            meta["page_display"] = None

        # stable chunk id (simplified version)
        meta["chunk_id"] = meta.get("chunk_id") or f"{meta.get('doc_name','doc')}_p{meta.get('page_display','?')}_c{i}"

        d.metadata = meta
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 5860


In [8]:
#text_chunks

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [19]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

/var/folders/g5/54vbnb5j1gdc46kk4fr0tvmr0000gn/T/ipykernel_65605/4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [13]:
#query_result


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [16]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [17]:
from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings, 
    index_name=index_name
)

In [18]:
#loading existing index

from langchain_pinecone import PineconeVectorStore
# we embed each chunk and upsert the embeddings into our Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name=index_name
)

In [19]:
docsearch

In [20]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [21]:
retrieved_docs = docsearch.similarity_search("What is diabetes?")

In [22]:
retrieved_docs

[Document(id='116955f8-e361-4dba-b050-6a208ac55ab6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page_label': '543', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,\nkidneys, nerves, heart, blood vessels, and other body\norgans. Hypoglycemia, or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia is'),
 Document(id='f3844b42-7633-432d-a6f0-dd3d53639b29', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page

In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.4, max_tokens=500)



In [24]:
import langchain
print(langchain.__version__)

1.2.4


In [32]:
from langchain_classic.chains import LLMChain


In [33]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assitant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [37]:
response = rag_chain.invoke({"input": "What is diabetes?"})
print(response["answer"])

Diabetes is a chronic condition where the body either does not produce enough insulin or the insulin produced does not work effectively, leading to high blood sugar levels, a condition known as hyperglycemia. If left untreated, diabetes can cause serious damage to various organs, including the eyes, kidneys, nerves, heart, and blood vessels. Early diagnosis and management are crucial to prevent complications.


In [3]:
# Add to your imports section (after existing imports)
from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

/Users/nihal/anaconda3/envs/llmapp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# HyDE Prompt - generates hypothetical medical document
hyde_prompt_template = """You are a medical expert. Write a detailed medical passage that would answer this question as it would appear in a medical textbook.

Question: {question}

Write a factual medical passage (3-4 sentences) using proper medical terminology:"""

hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template=hyde_prompt_template
)

In [6]:
# Initialize LLM for hypothesis generation (higher temperature for diversity)
from langchain_openai import ChatOpenAI
hyde_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=300)

# Create HyDE chain
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

In [22]:
# Test: Generate hypothesis for a sample question
test_question = "What are the symptoms of diabetes?"

print("Original Question:")
print(test_question)
print("\n" + "="*60)

# Generate hypothesis
hypothesis = hyde_chain.invoke({"question": test_question})

print("\nGenerated Hypothesis (HyDE):")
print(hypothesis)
print("="*60)

Original Question:
What are the symptoms of diabetes?


Generated Hypothesis (HyDE):
Diabetes mellitus is characterized by a spectrum of symptoms that can vary depending on the type and duration of the disease. Common clinical manifestations include polyuria (excessive urination), polydipsia (excessive thirst), and polyphagia (increased hunger), which arise due to hyperglycemia and its osmotic effects. Patients may also experience unexplained weight loss, fatigue, blurred vision, and delayed wound healing. In cases of diabetic ketoacidosis, particularly in Type 1 diabetes, additional symptoms such as abdominal pain, nausea, vomiting, and altered mental status may occur, necessitating urgent medical intervention.


In [25]:
# These cells from your trials.ipynb:

# 1. Load embeddings
embeddings = download_hugging_face_embeddings()

# 2. Connect to Pinecone
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings, 
    index_name="medicalbot"
)

# 3. Initialize LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4, max_tokens=300)

In [26]:
# Test question
question = "What causes chest pain?"

print(f"Question: {question}\n")
print("="*80)

# BASELINE: Retrieve using original question
print("\n🔵 BASELINE RETRIEVAL (Direct Query)")
print("-"*80)
baseline_docs = docsearch.similarity_search(question, k=3)

for i, doc in enumerate(baseline_docs, 1):
    page = doc.metadata.get("page_display", "?")
    print(f"\n[{i}] Page {page}:")
    print(doc.page_content[:200] + "...")

print("\n" + "="*80)

# HYDE: Generate hypothesis first, then retrieve
print("\n🟢 HYDE RETRIEVAL (Hypothetical Document)")
print("-"*80)

# Step 1: Generate hypothesis
hypothesis = hyde_chain.invoke({"question": question})
print(f"\nGenerated Hypothesis:\n{hypothesis}\n")
print("-"*80)

# Step 2: Retrieve using hypothesis
hyde_docs = docsearch.similarity_search(hypothesis, k=3)

for i, doc in enumerate(hyde_docs, 1):
    page = doc.metadata.get("page_display", "?")
    print(f"\n[{i}] Page {page}:")
    print(doc.page_content[:200] + "...")

print("\n" + "="*80)
print("\n📊 COMPARISON:")
baseline_pages = [doc.metadata.get("page_display") for doc in baseline_docs]
hyde_pages = [doc.metadata.get("page_display") for doc in hyde_docs]

print(f"Baseline pages: {baseline_pages}")
print(f"HyDE pages:     {hyde_pages}")
print(f"Pages overlap:  {set(baseline_pages) & set(hyde_pages)}")

Question: What causes chest pain?


🔵 BASELINE RETRIEVAL (Direct Query)
--------------------------------------------------------------------------------

[1] Page 209.0:
nary artery muscle spasm of insufficient duration or
intensity to cause an actual heart attack.
Causes and symptoms
Angina causes a pressing pain or sensation of heavi-
ness, usually in the chest area...

[2] Page 339.0:
the region of the aorta within the chest are called thoracic
aortic aneurysms. Aneurysms that occur in the part of the
aorta within the abdomen are called abdominal aortic
aneurysms.
Thoracic aortic a...

[3] Page 208.0:
Angina
Definition
Angina is pain, “discomfort,” or pressure localized
in the chest that is caused by an insufficient supply of
blood ( ischemia ) to the heart muscle. It is also some-
times characteri...


🟢 HYDE RETRIEVAL (Hypothetical Document)
--------------------------------------------------------------------------------

Generated Hypothesis:
Chest pain, or thoracic pain, can ari

In [27]:
def hyde_rag_pipeline(question: str, k: int = 5):
    """
    Complete HyDE RAG pipeline: Question → Hypothesis → Retrieve → Answer
    """
    print(f"❓ Question: {question}\n")
    
    # Step 1: Generate hypothesis
    print("🧠 Step 1: Generating hypothesis...")
    hypothesis = hyde_chain.invoke({"question": question})
    print(f"   Hypothesis: {hypothesis[:150]}...\n")
    
    # Step 2: Retrieve using hypothesis
    print(f"🔍 Step 2: Retrieving top-{k} documents using hypothesis...")
    retrieved_docs = docsearch.similarity_search(hypothesis, k=k)
    pages = [doc.metadata.get("page_display") for doc in retrieved_docs]
    print(f"   Retrieved pages: {pages}\n")
    
    # Step 3: Generate final answer
    print("💬 Step 3: Generating final answer...")
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    final_prompt = f"""You are a medical assistant. Based on the context below, answer the question concisely.

Context:
{context}

Question: {question}

Answer (2-3 sentences):"""
    
    answer_response = llm.invoke(final_prompt)
    answer = answer_response.content
    print(f"   Answer: {answer}\n")
    
    return {
        "question": question,
        "hypothesis": hypothesis,
        "retrieved_pages": pages,
        "answer": answer,
        "sources": retrieved_docs
    }

# Test the full pipeline
result = hyde_rag_pipeline("What are common symptoms of asthma?")

❓ Question: What are common symptoms of asthma?

🧠 Step 1: Generating hypothesis...
   Hypothesis: Asthma is a chronic inflammatory disorder of the airways characterized by recurrent episodes of wheezing, dyspnea (shortness of breath), chest tightne...

🔍 Step 2: Retrieving top-5 documents using hypothesis...
   Retrieved pages: [395.0, 394.0, 396.0, 394.0, 394.0]

💬 Step 3: Generating final answer...
   Answer: Common symptoms of asthma include wheezing, shortness of breath, coughing, and a feeling of tightness in the chest. In children, symptoms may also include itching on the back or neck. These symptoms can be triggered by factors such as cold air, exercise, stress, allergens, and respiratory infections.



In [28]:
# Test questions
test_questions = [
    "What is diabetes?",
    "What causes chest pain?",
    "What are symptoms of asthma?",
    "How is hypertension treated?",
    "What is appendicitis?"
]

comparison_results = []

for q in test_questions:
    print(f"\n{'='*80}")
    print(f"Testing: {q}")
    print(f"{'='*80}\n")
    
    # Baseline
    baseline_docs = docsearch.similarity_search(q, k=3)
    baseline_pages = [d.metadata.get("page_display") for d in baseline_docs]
    
    # HyDE
    hypothesis = hyde_chain.invoke({"question": q})
    hyde_docs = docsearch.similarity_search(hypothesis, k=3)
    hyde_pages = [d.metadata.get("page_display") for d in hyde_docs]
    
    # Compare
    overlap = len(set(baseline_pages) & set(hyde_pages))
    
    comparison_results.append({
        "question": q,
        "baseline_pages": baseline_pages,
        "hyde_pages": hyde_pages,
        "pages_overlap": overlap,
        "hypothesis": hypothesis[:100] + "..."
    })
    
    print(f"Baseline pages: {baseline_pages}")
    print(f"HyDE pages:     {hyde_pages}")
    print(f"Overlap:        {overlap}/3")
    print(f"Hypothesis:     {hypothesis[:150]}...")

# Show summary
print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}\n")
avg_overlap = sum(r["pages_overlap"] for r in comparison_results) / len(comparison_results)
print(f"Average page overlap: {avg_overlap:.1f}/3")
print(f"Different retrieval in: {sum(1 for r in comparison_results if r['pages_overlap'] < 3)}/{len(comparison_results)} questions")


Testing: What is diabetes?

Baseline pages: [543.0, 199.0, 542.0]
HyDE pages:     [543.0, 416.0, 416.0]
Overlap:        1/3
Hypothesis:     Diabetes mellitus is a chronic metabolic disorder characterized by hyperglycemia, which results from defects in insulin secretion, insulin action, or ...

Testing: What causes chest pain?

Baseline pages: [209.0, 339.0, 208.0]
HyDE pages:     [209.0, 421.0, 367.0]
Overlap:        1/3
Hypothesis:     Chest pain, or thoracalgia, can arise from a multitude of etiologies, broadly categorized into cardiac, pulmonary, gastrointestinal, musculoskeletal, ...

Testing: What are symptoms of asthma?

Baseline pages: [394.0, 395.0, 250.0]
HyDE pages:     [395.0, 395.0, 636.0]
Overlap:        1/3
Hypothesis:     Asthma is a chronic inflammatory disorder of the airways characterized by variable airflow obstruction and bronchial hyperresponsiveness. Common sympt...

Testing: How is hypertension treated?

Baseline pages: [293.0, 411.0, 217.0]
HyDE pages:     [293

In [29]:
# Questions with known correct pages
evaluation_questions = [
    {"question": "What is diabetes?", "expected_pages": [542, 543]},
    {"question": "What are symptoms of asthma?", "expected_pages": [395]},
]

print("RETRIEVAL QUALITY COMPARISON\n")
print(f"{'Question':<40} | {'Method':<10} | {'Found':<6} | {'Pages'}")
print("-" * 80)

for item in evaluation_questions:
    q = item["question"]
    expected = set(item["expected_pages"])
    
    # Baseline
    baseline_docs = docsearch.similarity_search(q, k=5)
    baseline_pages = set([d.metadata.get("page_display") for d in baseline_docs if d.metadata.get("page_display")])
    baseline_hit = len(baseline_pages & expected) > 0
    
    # HyDE
    hypothesis = hyde_chain.invoke({"question": q})
    hyde_docs = docsearch.similarity_search(hypothesis, k=5)
    hyde_pages = set([d.metadata.get("page_display") for d in hyde_docs if d.metadata.get("page_display")])
    hyde_hit = len(hyde_pages & expected) > 0
    
    print(f"{q:<40} | {'Baseline':<10} | {'✓' if baseline_hit else '✗':<6} | {sorted(baseline_pages)}")
    print(f"{'':<40} | {'HyDE':<10} | {'✓' if hyde_hit else '✗':<6} | {sorted(hyde_pages)}")
    print(f"{'':<40} | {'Expected':<10} | {'':<6} | {sorted(expected)}")
    print("-" * 80)

RETRIEVAL QUALITY COMPARISON

Question                                 | Method     | Found  | Pages
--------------------------------------------------------------------------------
What is diabetes?                        | Baseline   | ✓      | [199.0, 275.0, 278.0, 542.0, 543.0]
                                         | HyDE       | ✓      | [275.0, 276.0, 416.0, 543.0]
                                         | Expected   |        | [542, 543]
--------------------------------------------------------------------------------
What are symptoms of asthma?             | Baseline   | ✓      | [250.0, 393.0, 394.0, 395.0]
                                         | HyDE       | ✓      | [393.0, 394.0, 395.0, 396.0]
                                         | Expected   |        | [395]
--------------------------------------------------------------------------------
